# 基于Qwen2.5的LoRA微调

参考以下教程实现

[Qwen2.5-7B-Instruct Lora 微调](https://github.com/datawhalechina/self-llm?tab=readme-ov-file)

## Step1 导入相关包

In [ ]:
import os
os.environ["HF_HOME"] = '/root/autodl-tmp/huggingface'
os.environ["TRANSFORMERS_CACHE"] = '/root/autodl-tmp/huggingface'

In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

## Step2 加载数据集

In [ ]:
import json

from datasets import Dataset

# 数据集路径
input_file = "./dataset/chat_trans.jsonl"


def load_jsonl_data(file_path):
    json_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            json_data.append(json.loads(line.strip()))
    return json_data


# 将数据加载为 Dataset 格式
data = load_jsonl_data(input_file)
dataset = Dataset.from_list(data)
dataset

## Step3 数据预处理

In [ ]:
# 这里使用的是AUTODL上的镜像，直接加载镜像里的模型
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct", use_fast=False, trust_remote_code=True)
#tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct/', use_fast=False, trust_remote_code=True)

In [ ]:
def process_func(example):
    MAX_LENGTH = 384
    input_ids_batch, attention_mask_batch, labels_batch = [], [], []

    for idx, question in enumerate(example['question']):
        instruction = tokenizer(f"<|im_start|>system\n宜家你系讲信宜话个智能助手--阿信<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n",
                                add_special_tokens=False)
        response = tokenizer(f"{example['answer'][idx]}", add_special_tokens=False)
        input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
        attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
        labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

        # 截断操作
        if len(input_ids) > MAX_LENGTH:
            input_ids = input_ids[:MAX_LENGTH]
            attention_mask = attention_mask[:MAX_LENGTH]
            labels = labels[:MAX_LENGTH]

        # 添加到批量列表
        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)
        labels_batch.append(labels)

    return {
        "input_ids": input_ids_batch,
        "attention_mask": attention_mask_batch,
        "labels": labels_batch
    }

In [ ]:
tokenized_dataset = dataset.map(process_func, batched=True)

In [ ]:
# 半精度加载模型，减少内存使用
# model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct/', device_map="auto", torch_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", device_map="auto", torch_dtype=torch.bfloat16)

In [ ]:
model.enable_input_require_grads()

In [ ]:
model.dtype

## Step5 定义lora参数

In [ ]:
from peft import LoraConfig, TaskType

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1  # Dropout 比例
)

In [ ]:
from peft import get_peft_model

# # 应用 LoRA 配置到模型
model = get_peft_model(model, config)
config

In [ ]:
# 查看训练参数
model.print_trainable_parameters()

## Step6 配置训练参数

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./output/Qwen2.5_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

## Step7 创建Trainer

In [ ]:
from transformers import DataCollatorForSeq2Seq, Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step8 训练模型

In [ ]:
trainer.train()

## Step9 评估模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/root/autodl-tmp/qwen/Qwen2.5-7B-Instruct/'
lora_path = './output/Qwen2.5_instruct_lora/checkpoint-10'  # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)


In [ ]:
prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "宜家你系讲信宜话个助手--阿信"}, {"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Step10 保存模型